In [1]:
import os
import pandas as pd
import datetime as dt
import time
import calendar
import numpy as np
from statistics import mean

from uce_resources import get_site_id, get_mms_data, get_applied_forecast, get_prices, get_green_tariff
from uce_resources import get_forecast, make_results

In [2]:
target_year = 2022
target_month = 1


forecasts_types = ['real', 'naive', 'zero', '1_dah', 'pro', 'restored']

In [3]:
sites_list = ['Myroliubivka', 'Novokondakove', 'Kyselivka', 'Poniativka', 'Bar', 'Verkhivka', 'Balky', 
              'Sharhorod_1', 'Kostohryzove', 'Chechelnyk_1', 'Chechelnyk_2', 'Stanislavchyk', 'Kopaihorod', 'Cherniatka', 
              'Bilozerka', 'Hlybochok_1', 'Hlybochok_2.1', 'Hlybochok_2.2', 'Mykolaivka', 'Komyshany_1', 'Komyshany_2', 
              'Dibrovka', 'Velihen', 'Veliton', 'Mala_Lepetykha', 'Rubanivka', 'Oleshky_2', 'Oleshky_1',
              'Afanasiivka', 'Bazaltova', 'Liubymivka', 'Balivka', 'Pohrebyshche', 'Bilashky', 'Yelanets_1',
              'Yelanets_2', 'Inhulets_1', 'Inhulets_2', 'Bereznehuvate', 'Teplychna', 'Solone', 'Stepnohirsk', 
              'Vasylivka', 'Kachkarivka', 'Hnatkiv', 'Porohy', 'Kulevcha']
              
# sites_list = ['Myroliubivka']
sites_data = dict.fromkeys(sites_list)
print(sites_data)
print(len(sites_data.keys()))

{'Myroliubivka': None, 'Novokondakove': None, 'Kyselivka': None, 'Poniativka': None, 'Bar': None, 'Verkhivka': None, 'Balky': None, 'Sharhorod_1': None, 'Kostohryzove': None, 'Chechelnyk_1': None, 'Chechelnyk_2': None, 'Stanislavchyk': None, 'Kopaihorod': None, 'Cherniatka': None, 'Bilozerka': None, 'Hlybochok_1': None, 'Hlybochok_2.1': None, 'Hlybochok_2.2': None, 'Mykolaivka': None, 'Komyshany_1': None, 'Komyshany_2': None, 'Dibrovka': None, 'Velihen': None, 'Veliton': None, 'Mala_Lepetykha': None, 'Rubanivka': None, 'Oleshky_2': None, 'Oleshky_1': None, 'Afanasiivka': None, 'Bazaltova': None, 'Liubymivka': None, 'Balivka': None, 'Pohrebyshche': None, 'Bilashky': None, 'Yelanets_1': None, 'Yelanets_2': None, 'Inhulets_1': None, 'Inhulets_2': None, 'Bereznehuvate': None, 'Teplychna': None, 'Solone': None, 'Stepnohirsk': None, 'Vasylivka': None, 'Kachkarivka': None, 'Hnatkiv': None, 'Porohy': None, 'Kulevcha': None}
47


# Data preparation section

In [4]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool

do_settings = {'host': 'db-postgresql-do-pv-do-user-1646238-0.db.ondigitalocean.com',
               'port': 25060,
               'database': 'pv-analytics', 
               'user': 'pv-analytics-master',
               'password': 'dzn9dyvnkp2llz3k'}

url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(do_settings['user'], do_settings['password'], do_settings['host'], do_settings['port'], do_settings['database'])
engine = create_engine(url, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\postgresql\base.py:3802: SAWarning: Did not recognize type 'point' of column 'location'
  util.warn(


In [5]:
with engine.connect() as connection:
    
    prices = get_prices(target_year, target_month, connection, metadata.tables['electricity_market_prices'], currency='UAH')

prices

,dam,imsp,positive_unbalance,negative_unbalance
2021-12-31 22:30:00,1.70000,0.09074,0.08620,1.78500
2021-12-31 23:30:00,1.04996,0.00001,0.00001,1.10246
2022-01-01 00:30:00,1.04996,0.00001,0.00001,1.10246
2022-01-01 01:30:00,1.04999,0.00001,0.00001,1.10249
2022-01-01 02:30:00,1.04996,0.00001,0.00001,1.10246
...,...,...,...,...
2022-01-20 17:30:00,3.24995,3.24995,3.08745,3.41245
2022-01-20 18:30:00,3.19900,3.19900,3.03905,3.35895
2022-01-20 19:30:00,3.03949,0.54000,0.51300,3.19146
2022-01-20 20:30:00,2.80000,0.00001,0.00001,2.94000


In [6]:
#prices.to_excel('results/{}-{:0>2}/prices_{}_{}_1-31.xlsx'.format(target_year, target_month, target_year, target_month))

In [7]:
target_dates = pd.date_range(start=prices.index.date.min() + dt.timedelta(days=1),
                             end=prices.index.date.max(), 
                             freq='D').to_pydatetime()

In [8]:

with engine.connect() as connection:
        
    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)
                
        site_data['green_tariff'] = get_green_tariff(site_data['site_id'], dt.date(year=target_year, month=target_month, day=1),
                                                     connection, metadata.tables['green_tariffs'], currency='UAH')
        print('Green tariff: {}'.format(site_data['green_tariff']))
        
        mms_data, site_data['mms_version'] = get_mms_data(site_data['site_id'], 
                                                          target_year, target_month, 
                                                          connection, metadata.tables['mms_data'], include_prev=True,)
        # print(mms_data)
        print('MMS data | {} version | of | {} records |'.format(site_data['mms_version'], len(mms_data)))
        applied_forecast = get_applied_forecast(site_data['site_id'], target_year, target_month, 
                                                connection=connection, db_table=metadata.tables['forecasts_applied'])
        print('Forecast data of | {} records |'.format(len(applied_forecast)))
        #print(applied_forecast)
        
        site_data['real_forecast_data'] = pd.concat([mms_data, applied_forecast], axis=1, join='inner')
        print('Real forecast data prepared')

        site_data['zero_forecast_data'] = pd.concat([mms_data, applied_forecast * 0], axis=1, join='inner')
        print('Zero forecast data prepared')

        naive_forecast_data = pd.concat([mms_data, mms_data.shift(48)], axis=1, join='inner').dropna(axis=0, how='any')
        naive_forecast_data.columns = ['yield [kWh]', 'forecast [kWh]']
        naive_forecast_data['forecast [kWh]'] = naive_forecast_data['forecast [kWh]'].astype(int)
        site_data['naive_forecast_data'] = naive_forecast_data
        print('Naive forecast data prepared')

        one_dah_forecast = get_forecast(site_data['site_id'], target_dates, '1dah', connection, metadata) * 1000
        one_dah_forecast = pd.concat([mms_data, one_dah_forecast.round()], axis=1, join='inner')
        # site_data['1dah_forecast'].to_csv('data/uce_update_frequency/separately/' + site + '/' + site + '_1dah_forecast.csv')
        site_data['1_dah_forecast_data'] = one_dah_forecast
        print('1dah forecast data prepared')

        pro_forecast = get_forecast(site_data['site_id'], target_dates, 'pro', connection, metadata) * 1000
        pro_forecast = pd.concat([mms_data, pro_forecast.round()], axis=1, join='inner')
        # site_data['pro_forecast'].to_csv('data/uce_update_frequency/separately/{}/{}_pro_forecast.csv'.format(site, site))
        site_data['pro_forecast_data'] = pro_forecast
        print('Pro forecast data prepared')

        restored_forecast = get_forecast(site_data['site_id'], target_dates, 'restored', connection, metadata) * 1000
        restored_forecast = pd.concat([mms_data, restored_forecast.round()], axis=1, join='inner')
        # site_data['pro_forecast'].to_csv('data/uce_update_frequency/separately/{}/{}_pro_forecast.csv'.format(site, site))
        site_data['restored_forecast_data'] = restored_forecast
        print('Restored forecast data prepared')   
        
        sites_data.update({site: site_data})
        end = time.time()

        print('Processing took {} seconds'.format(round(end - start, 2)))

--------------------------------------------------
Myroliubivka
Green tariff: 4.1268
MMS data | v1 version | of | 1224 records |
Forecast data of | 576 records |
Real forecast data prepared
Zero forecast data prepared
Naive forecast data prepared
1dah
2022-01-01
SolarGIS_hourly_3_Myroliubivka_20211231_20220114_202112311403.csv
2022-01-02
SolarGIS_hourly_3_Myroliubivka_20220101_20220115_202201011402.csv
2022-01-03
SolarGIS_hourly_3_Myroliubivka_20220102_20220116_202201021402.csv
2022-01-04
SolarGIS_hourly_3_Myroliubivka_20220103_20220117_202201031402.csv
2022-01-05
SolarGIS_hourly_3_Myroliubivka_20220104_20220118_202201041402.csv
2022-01-06
SolarGIS_hourly_3_Myroliubivka_20220105_20220119_202201051402.csv
2022-01-07
SolarGIS_hourly_3_Myroliubivka_20220106_20220120_202201061402.csv
2022-01-08
SolarGIS_hourly_3_Myroliubivka_20220107_20220121_202201071402.csv
2022-01-09
SolarGIS_hourly_3_Myroliubivka_20220108_20220122_202201081402.csv
2022-01-10
SolarGIS_hourly_3_Myroliubivka_20220109_2022

In [ ]:
# site = 'Myroliubivka'
# site_data = None
# # print(site_data)
# site_data = sites_data['Myroliubivka']['zero_forecast_data'].copy()
# # print(site_data)
# site_data.columns = ['yield [kWh]', 'zero_forecast [kWh]']
# # site_data['naive_forecast [kWh]'] = sites_data[site]['naive_forecast_data']['forecast [kWh]']
# site_data['real_forecast [kWh]'] = sites_data[site]['real_forecast_data']['forecast [kWh]']
# site_data['1_dah_forecast [kWh]'] = sites_data[site]['1_dah_forecast_data']['forecast [kWh]']
# site_data['pro_forecast [kWh]'] = sites_data[site]['pro_forecast_data']['forecast [kWh]']
# site_data['restored_forecast [kWh]'] = sites_data[site]['restored_forecast_data']['forecast [kWh]']
# # site_data = site_data.loc[site_data.index.day == 18]
# print(site_data)
# file_name = 'results/{}_data_{}_{}.xlsx'.format(site.lower(), target_year, target_month)
# print(file_name)
# writer = pd.ExcelWriter(file_name, engine="openpyxl")
# site_data.to_excel(writer, 'data')
# writer.save()
# print('Writting ok!')

## Unbalance cost estimations

In [ ]:
columns = ['site', 'legal_entity', 'first_date', 'last_date', 'number_of_values [records]', 'yield_data_version',
            'yielded [kWh]', 'forecast_type', 'forecasted [kWh]', 
            'green_tariff [UAH]', 'revenue [UAH]', 
            'error_u [kWh]', 'error_u [%]',
            'max_energy [kWh]', 'max_forecast [kWh]', 'max_error [kWh]',
            'mean_absolute_error [kWh]', 'median_absolute_error [kWh]', 
            'mean_square_error [kWh]', 'root_mean_square_error [kWh]', 'R^2 score',
            'dropped by alpha_u [records]', 'dropped by alpha_u [%]',
            'error_u (excess) [kWh]', 'error_u (excess) [%]',
            'error_u (shortage) [kWh]', 'error_u (shortage) [%]', 
            'cieq_641_rule (excess) [UAH]', 'cieq_641_rule (excess) [%]',
            'cieq_641_rule (shortage) [UAH]', 'cieq_641_rule (shortage) [%]',
            'cieq_641_rule (net) [UAH]', 'cieq_641_rule (net) [%]', 
            'imsp_avg_641_rule [UAH/MWh]',
            'cieq_641_rule* [UAH]', 'cieq_641_rule* [%]', 
            'imsp_avg_641_rule* [UAH/MWh]']

### Daily results

In [ ]:
daily_indexes = list()

for day in range(1, calendar.monthrange(target_year, target_month)[-1] + 1):
    start = dt.datetime(year=target_year, month=target_month, day=day, hour=0, minute=30)
    end = dt.datetime(year=target_year, month=target_month, day=day, hour=23, minute=30)
    index_in_kyiv = pd.date_range(start=start, end=end, freq='1H', tz='europe/kiev')
    index_in_utc = index_in_kyiv.tz_convert('utc').tz_localize(None)
    daily_indexes.append(index_in_utc)

print(len(daily_indexes))

In [ ]:
print(daily_indexes[0])

In [ ]:
results_real = pd.DataFrame(columns=columns)
results_naive = pd.DataFrame(columns=columns)
results_zero = pd.DataFrame(columns=columns)
results_1_dah = pd.DataFrame(columns=columns)
results_pro = pd.DataFrame(columns=columns)
results_restored = pd.DataFrame(columns=columns)

for site in sites_data.keys():
    
    
    for index in daily_indexes:
        # print(sites_data[site]['real_forecast_data'])
        result_real = make_results(sites_data[site], 'real', prices, index)
        #print(result_real)

        result_naive = make_results(sites_data[site], 'naive', prices, index)      
        #print(result_naive)

        result_zero = make_results(sites_data[site], 'zero', prices, index)      
        #print(result_zero)

        result_1_dah = make_results(sites_data[site], '1_dah', prices, index)      
        #print(result_1_dah)

        result_pro = make_results(sites_data[site], 'pro', prices, index)      
        #print(result_pro)

        result_restored = make_results(sites_data[site], 'restored', prices, index)
        #print(result_restored)

        if not result_real is None:
            results_real = results_real.append(result_real, ignore_index=True)

        if not result_naive is None:
            results_naive = results_naive.append(result_naive, ignore_index=True)
        
        if not result_zero is None:
            results_zero = results_zero.append(result_zero, ignore_index=True)
            
        if not result_1_dah is None:
            results_1_dah = results_1_dah.append(result_1_dah, ignore_index=True)

        if not result_pro is None:
            results_pro = results_pro.append(result_pro, ignore_index=True)

        if not result_restored is None:
            results_restored = results_restored.append(result_restored, ignore_index=True)


    sites_data[site]['results_real'] = results_real
    sites_data[site]['results_naive'] = results_naive
    sites_data[site]['results_zero'] = results_zero
    sites_data[site]['results_1_dah'] = results_1_dah
    sites_data[site]['results_pro'] = results_pro
    sites_data[site]['results_restored'] = results_restored

    print(f'{site} - Results daily: Ok!')

In [ ]:
# sites_data['Pohrebyshche']['results_real']

In [ ]:
from uce_resources import save_results, format_excel

results_daily = pd.concat([results_real, results_naive, results_zero, results_1_dah, results_pro, results_restored], axis=0)

with pd.ExcelWriter('results/{}-{:0>2}/uce_r_daily_{}_{}_{}_UAH.xlsx'.format(target_year, target_month, target_year, target_month, '1-18'), engine="openpyxl") as writer:
    results_daily.to_excel(writer, 'results_daily')

# writer.save()
#format_excel(writer, results_daily).save()
 
print('Saving results: ok!')


In [ ]:
print(sites_data['Myroliubivka']['results_real']['max_energy [kWh]'].max())
print(sites_data['Myroliubivka']['results_real']['max_forecast [kWh]'].max())